
# Efficient Frontier (Mean-Variance)

The Markowitz framework finds portfolios that minimize variance for a given
expected return.

Portfolio return and variance:

```{math}
\mathbb{E}[R_p] = w^T \mu, \quad \sigma_p^2 = w^T \Sigma w
```

Optimization problem:

```{math}
\min_{w} \; w^T \Sigma w \quad \text{s.t.} \quad w^T \mu = r, \; \mathbf{1}^T w = 1
```

Sharpe ratio:

```{math}
\text{Sharpe} = \frac{\mathbb{E}[R_p] - r_f}{\sigma_p}
```

The **efficient frontier** is the set of portfolios with the lowest risk for each
return level. The **tangency portfolio** maximizes the Sharpe ratio.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import HTML

def show_plot(fig):
    return HTML(fig.to_html(include_plotlyjs=False, full_html=False))

np.random.seed(7)



## Synthetic Inputs
We use a synthetic mean and covariance to illustrate the geometry.

In [ ]:
n_assets = 5
mean_returns = np.array([0.08, 0.10, 0.12, 0.07, 0.09])
A = np.random.randn(n_assets, n_assets)
cov = A @ A.T
cov = cov / np.max(cov) * 0.04

n_portfolios = 5000
weights = np.random.random((n_portfolios, n_assets))
weights = weights / weights.sum(axis=1, keepdims=True)

portfolio_returns = weights @ mean_returns
portfolio_vol = np.sqrt(np.einsum('ij,jk,ik->i', weights, cov, weights))
risk_free = 0.02
sharpe = (portfolio_returns - risk_free) / portfolio_vol

max_sharpe_idx = np.argmax(sharpe)
min_vol_idx = np.argmin(portfolio_vol)



## Approximate Frontier (Sampling)
We approximate the frontier by taking the minimum volatility portfolio in
return bins. This is a quick visualization, not an exact optimizer.

In [ ]:
bins = np.linspace(portfolio_returns.min(), portfolio_returns.max(), 25)
frontier_r = []
frontier_v = []
for i in range(len(bins) - 1):
    mask = (portfolio_returns >= bins[i]) & (portfolio_returns < bins[i + 1])
    if np.any(mask):
        idx = np.argmin(portfolio_vol[mask])
        sel_vol = portfolio_vol[mask][idx]
        sel_ret = portfolio_returns[mask][idx]
        frontier_v.append(sel_vol)
        frontier_r.append(sel_ret)
frontier_v = np.array(frontier_v)
frontier_r = np.array(frontier_r)



## Efficient Frontier Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=portfolio_vol,
    y=portfolio_returns,
    mode='markers',
    marker=dict(color=sharpe, colorscale='Viridis', showscale=True),
    name='Portfolios'
))
fig.add_trace(go.Scatter(
    x=frontier_v,
    y=frontier_r,
    mode='lines+markers',
    name='Approx Frontier'
))
fig.add_trace(go.Scatter(
    x=[portfolio_vol[max_sharpe_idx]],
    y=[portfolio_returns[max_sharpe_idx]],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Max Sharpe'
))
fig.add_trace(go.Scatter(
    x=[portfolio_vol[min_vol_idx]],
    y=[portfolio_returns[min_vol_idx]],
    mode='markers',
    marker=dict(color='black', size=9),
    name='Min Vol'
))
fig.update_layout(
    title='Efficient Frontier (Sampled)',
    xaxis_title='Volatility',
    yaxis_title='Expected Return',
    template='plotly_white'
)
show_plot(fig)
